In [ ]:
import agents
from html2shot_async import html2shot
from vendors.openai import gpt4o
from vendors.google import gemini
from PIL import Image
from agents import *
from smart_blocker import *
agents.BACKBONE = gemini

In [ ]:
html = generater.infer(imgs=[image])

In [ ]:
path = "xx"
def image2md5(image):
    image_bytes = io.BytesIO()
    image.save(image_bytes, format='PNG')
    image_data = image_bytes.getvalue()
    md5_hash = hashlib.md5(image_data)
    md5_hex = md5_hash.hexdigest()
    return str(md5_hex)
md5 = image2md5(image=image)
imagehtml = await html2shot(path,html,md5,111)

In [ ]:
# ，
from tqdm import tqdm
from agents import *
from smart_blocker import *
save_path=f'xx'
base_path = save_path
agentGenerate = AgentGenerate()

def generate_module_code(plans):
    md5 = image2md5(image)
    codes_plans = []
    for index,plan in tqdm(enumerate(plans,start=1),total=len(plans)):
        module_image = crop_image(image,plan)
        path = save_path + f"/{md5}"
        os.makedirs(path, exist_ok=True)
        path += f"/{index}.png"

        print(path)
        module_image.save(path)

        # code = agentGenerate.infer(image1=image,image2=module_image,text=json.dumps(plan))
        code = agentGenerate.infer([module_image])
        code=remove_code_markers(code)
        print(code)

        image_pre = html2shot(base_path, code,md5,index)

        codes_plans.append({"module_position":plan,"module_code":code})
    
    return codes_plans
image = Image.open("XX/image_27.png")
# 
plans = blocker(image)
codes = generate_module_code(plans)

In [ ]:
def image2md5(image):
    image_bytes = io.BytesIO()
    image.save(image_bytes, format='PNG')
    image_data = image_bytes.getvalue()
    md5_hash = hashlib.md5(image_data)
    md5_hex = md5_hash.hexdigest()
    return str(md5_hex)
# 
def absolute_assemble(image,code_plans):
    md5 = image2md5(image)

    html_image = []

    code_abss = []
    for node in code_plans:
        bbox = node['module_position']
        code_abs = f'<div style="position: absolute; overflow: hidden; border: 1px solid blue; left: {round(bbox[0] * image.width)}px; top: {round(bbox[1] * image.height)}px; width: {round((bbox[2] - bbox[0]) * image.width)}px; height: {round((bbox[3] - bbox[1]) * image.height)}px;">{"".join(node["module_code"])}</div>'
        code_abss.append(code_abs)

    html = """<html>
    <body>
        """ + '\n'.join(code_abss) + """
    </body>
</html>"""

    # imageofhtml = html2shot(base_path, html_content=html,md5=md5,index=1000)
    # html_image.append({"html":html,"image":imageofhtml})
    return html


html1code = absolute_assemble(image,codes)
md5 = image2md5(image=image)
imagejuedui = await html2shot(save_path,html1code,md5,1000)

In [ ]:
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
API_KEY_GEMINI ="xx-xx"
#  HTML 
def extract_html_from_response(response):
    try:
        if not response.candidates:
            print("No candidates found in response.")
            return None
        #  content 
        candidate = response.candidates[0]
        if 'content' not in candidate:
            print("No content found in candidate.")
            return None
        html_content = candidate.content.parts[0].text
        if html_content.startswith("```html"):
            html_content = html_content[7:]
        if html_content.endswith("```"):
            html_content = html_content[:-3]
        return html_content.strip()
    except Exception as e:
        print("Error extracting HTML:", e)
        return None


genai.configure(api_key=API_KEY_GEMINI)
def gemini(prompt, image, text):
    model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")
    # 
    try:
        response = model.generate_content([prompt] +image +[text])
        # prompt="Discribe this image."
        # response = model.generate_content([prompt,image])
        print(response)
        html_content = extract_html_from_response(response)
        return html_content
    except Exception as e:
        print("Error during content generation:", e)

In [ ]:
wenben

In [ ]:
refineimage = await html2shot(path,refinehtml,md5,1001)

In [ ]:
wenben

In [ ]:
imagehtml = await html2shot(path,newhtml,md5,99)

In [ ]:
newhtml

In [ ]:
def gemini(prompt, image, text):
    model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")
    # 
    try:
        response = model.generate_content([prompt]+image+[text])
        # prompt="Discribe this image."
        # response = model.generate_content([prompt,image])
        print(response)
        html_content = extract_html_from_response(response)
        return html_content
    except Exception as e:
        print("Error during content generation:", e)

In [ ]:
imagehtml = await html2shot(path,newhtml2,md5,100)

In [ ]:
import sys
from pathlib import Path
from PIL import Image
import json
import os
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
# 
sys.path.append("XX")

# 
from evaluation.mrweb.emd_similarity import emd_similarity
from evaluation.mrweb.study import CLIPScorer, LPIPSScorer, ssim_score, psnr_score, mae_score


def evaluate_images(img1, img2):
    """
    ，。
    """
    try:
        clip_scorer = CLIPScorer(model_name='ViT-B-32-quickgelu', pretrained='openai')
        lpips_scorer = LPIPSScorer(net='vgg')

        clip_score = clip_scorer.score(img1, img2)
        lpips_score = lpips_scorer.score(img1, img2)
        ssim_value = ssim_score(img1, img2)
        psnr_value = psnr_score(img1, img2)
        mae_value = mae_score(img1, img2)
        emd_score = emd_similarity(img1, img2, max_size=96, mode="RGB")

        return {
            "CLIP Score": clip_score,
            "LPIPS Score": lpips_score,
            "SSIM": ssim_value,
            "PSNR": psnr_value,
            "MAE": mae_value,
            "EMD Similarity": emd_score
        }
    except Exception as e:
        print(f"Error during evaluation: {e}")
        return None


def evaluate_all(ref_image, candidate_images):
    """
    ，。
    """
    results = []
    for idx, cand_image in enumerate(candidate_images):
        try:
            print(f"Evaluating candidate {idx + 1}/{len(candidate_images)}...")
            scores = evaluate_images(ref_image, cand_image)
            if scores:
                results.append({
                    "candidate_index": idx,
                    "scores": scores
                })
                # 
                print(f"Results for candidate {idx}:\n" + "\n".join(
                    [f"{metric}: {score}" for metric, score in scores.items()]))
            else:
                print(f"Failed to evaluate candidate {idx}")
        except Exception as e:
            print(f"Error evaluating candidate {idx}: {e}")

    
    return results
